# Box matching test

We match example "default boxes" with "ground truth boxes", and calculate matches

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
np.random.seed(seed=0)

from utils import iou

## Constants

In [2]:
IOU_THRESHOLD = 0.5
BATCH_SIZE = 2
NUM_PRIORS = 2
NUM_GTS = 2
NUM_FRAMES = 2

## Single frame case

In [ ]:
priors_arr = np.array([[1.0, 1.0, 2.0, 2.0], [0.0, 0.0, -1.0, -1.0], [0.0, 0.0, 2.0, 1.5], [0.0, 0.0, 1.5, 2.0], [0.0, 0.0, 0.5, 0.5], [2.5, 2.5, 3.0, 3.0]])
gts_arr = np.array([[0.0, 0.0, 2.0, 2.0], [2.0, 2.0, 3.0, 3.0]])


priors = pd.DataFrame(priors_arr, columns=['x0', 'y0', 'x1', 'y1'])
priors['prior_id'] = range(len(priors))
#priors['class'] = [1, 0, 1, 0, 1, 0]
#priors['logits'] = [[0, 1], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0]]
priors['temp'] = True

ground_truth_boxes = pd.DataFrame(gts_arr, columns=['x0', 'y0', 'x1', 'y1'])
ground_truth_boxes['gt_id'] = range(len(gts))
#ground_truth_boxes['class'] = [1, 0]
ground_truth_boxes['temp'] = True

### Calculate matches

In [ ]:
associations = pd.merge(priors, ground_truth_boxes, how='outer', suffixes=('_prior', '_gt'), on='temp')
associations = associations.drop(columns=['temp'])
associations['iou'] = iou(associations)
#associations['match'] = (associations['iou'] > IOU_THRESHOLD) & (associations['class_prior'] == associations['class_gt'])

### Select priors for ground truth box(es) with highest overlap

In [ ]:
best_priors = associations.iloc[associations.groupby('prior_id').apply(lambda g: g.iou.idxmax())]

### Hard negative mining

In [ ]:
TODO

## Batch case

In [ ]:
# TODO: create actual ground truth boxes, 

priors_arr = np.random.random_sample((BATCH_SIZE * NUM_PRIORS, 4))
priors = pd.DataFrame(priors_arr, columns=['x0', 'y0', 'x1', 'y1'])


for i in range(NUM_FRAMES):
    priors.loc[priors.index[i*NUM_PRIORS:(i+1)*NUM_PRIORS], 'frame_id'] = i
    priors.loc[priors.index[i*NUM_PRIORS:(i+1)*NUM_PRIORS], 'prior_id'] = range(NUM_PRIORS)


#gts_arr = np.random.sample((BATCH_SIZE * NUM_GTS, 4))
ground_truth_boxes = pd.DataFrame(priors_arr, columns=['x0', 'y0', 'x1', 'y1'])


for i in range(NUM_FRAMES):
    ground_truth_boxes.loc[ground_truth_boxes.index[i*NUM_GTS:(i+1)*NUM_GTS], 'frame_id'] = i
    ground_truth_boxes.loc[ground_truth_boxes.index[i*NUM_GTS:(i+1)*NUM_GTS], 'gt_id'] = range(NUM_GTS)


In [ ]:
associations = pd.merge(priors, ground_truth_boxes, how='outer', suffixes=('_prior', '_gt'), on='frame_id')
associations['iou'] = iou(associations)

## Real data case
TODO

## Non-maximum suppression
This happes after predictions are done with the model. Only the best prediction for each gt box remains.

In [3]:
def array_to_df(arr, classes, arr_type):
    assert(arr_type in ['prior', 'prediction', 'gt'])
    df = pd.DataFrame(arr.reshape((BATCH_SIZE * NUM_PRIORS, 4)), columns=['x0', 'y0', 'x1', 'y1'])

    for i in range(NUM_FRAMES):
        df.loc[df.index[i*NUM_PRIORS:(i+1)*NUM_PRIORS], 'frame_id'] = i
        df.loc[df.index[i*NUM_PRIORS:(i+1)*NUM_PRIORS], arr_type + '_id'] = range(NUM_PRIORS)

    df['class_' + arr_type] = classes.argmax(axis=1)
    return df


def get_associations(box_predictions_arr, class_predictions, box_gts_arr, class_gts):
    pred_type = 'prediction'
    gt_type = 'gt'
    
    prediction_df = array_to_df(box_predictions_arr, class_predictions, pred_type)

    gt_df = array_to_df(box_gts_arr, class_gts, gt_type)

    associations = pd.merge(prediction_df, gt_df, how='outer', suffixes=('_' + pred_type, '_' + gt_type), on='frame_id')
    associations['iou'] = iou(associations)
    associations['match'] = (associations['iou'] > IOU_THRESHOLD) & (associations['class_' + pred_type] == associations['class_' + gt_type])
    return associations

In [4]:
NUM_CLASSES = 3

ohe = OneHotEncoder()
ohe.fit(np.array(range(NUM_CLASSES)).reshape(-1, 1))

box_predictions = np.random.random_sample((BATCH_SIZE, NUM_GTS, 4))
class_predictions = ohe.transform(np.random.randint(NUM_CLASSES, size=(BATCH_SIZE, NUM_PRIORS)).reshape((-1, 1))).toarray()
box_gts = box_predictions
class_gts = class_predictions

associations = get_associations(box_predictions, class_predictions, box_gts, class_gts)

# Find best matching prediction to single gt
associations.iloc[associations[associations['frame_id'] == 0].groupby('gt_id').apply(lambda f: f.iou.idxmax())]

# TODO: do this to all gts

/home/CW01/uib10737/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
